# 必要なライブラリのインポート

In [15]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
 
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
import pandas as pd

# データセットの読み込み

In [16]:
wine = load_wine()
pd.DataFrame(wine.data, columns=wine.feature_names)

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0


In [17]:
wine_data = wine.data[0:130]
wine_target = wine.target[0:130]

In [18]:
train_X, test_X, train_Y, test_Y = train_test_split(wine_data, wine_target, test_size = 0.25)

# PyTorchへのテンソル（多次元行列）への変換

In [19]:
train_X = torch.from_numpy(train_X).float()
train_Y = torch.from_numpy(train_Y).long()
test_X = torch.from_numpy(test_X).float()
test_Y = torch.from_numpy(test_Y).long()

In [20]:
train=TensorDataset(train_X, train_Y)
train[0]

(tensor([1.4380e+01, 3.5900e+00, 2.2800e+00, 1.6000e+01, 1.0200e+02, 3.2500e+00,
         3.1700e+00, 2.7000e-01, 2.1900e+00, 4.9000e+00, 1.0400e+00, 3.4400e+00,
         1.0650e+03]),
 tensor(0))

In [21]:
train_loader = DataLoader(train, batch_size=15, shuffle=True)

# モデルの定義

In [22]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(13, 128)
        self.fc2 = nn.Linear(128, 2)
         
    def forward(self, x):
        x = F.relu(self.fc1(x)) # ReLU: max(x, 0)
        x = self.fc2(x)
        return F.log_softmax(x, dim=0)

In [23]:
model = Net()

# トレーニング

In [24]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
for epoch in range(500): #学習回数500回
    total_loss = 0
     
    for train_x, train_y in train_loader:
        train_x, train_y = Variable(train_x), Variable(train_y)
        optimizer.zero_grad()
        output = model(train_x)
        loss = criterion(output, train_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() #loss.data[0]で記述するとPyTorch0.5以上ではエラーが返る
     
    if (epoch+1)%60 == 0:
        print(epoch+1, total_loss)

60 4.852030992507935
120 4.852030992507935
180 4.852030992507935
240 4.852030992507935
300 4.852030992507935
360 4.852030992507935
420 4.852030992507935
480 4.852030992507935


# 精度計算

In [25]:
test_x, test_y = Variable(test_X), Variable(test_Y)
result = torch.max(model(test_x).data, 1)[1]
accuracy = sum(test_y.data.numpy() == result.numpy()) / len(test_y.data.numpy())
accuracy

0.7575757575757576

In [26]:
model(test_x).data

tensor([[-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965],
        [-3.4965, -3.4965]])

# ニューラルネットワークの多層化（ディープラーニング）による精度検証

In [27]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(13, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 128)
        self.fc4 = nn.Linear(128, 128)
        self.fc5 = nn.Linear(128, 128)
        self.fc6 = nn.Linear(128, 2)
         
    def forward(self, x):
        x = F.relu(self.fc1(x)) # ReLU: max(x, 0)
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return F.log_softmax(x, dim=0)

In [28]:
model = Net()

## トレーニング

In [29]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
for epoch in range(500): #学習回数500回
    total_loss = 0
     
    for train_x, train_y in train_loader:
        train_x, train_y = Variable(train_x), Variable(train_y)
        optimizer.zero_grad()
        output = model(train_x)
        loss = criterion(output, train_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() #loss.data[0]で記述するとPyTorch0.5以上ではエラーが返る
     
    if (epoch+1)%60 == 0:
        print(epoch+1, total_loss)

60 1.9004581570625305
120 1.1381404139101505
180 1.3101055473089218
240 1.2716394439339638
300 0.9724925558548421
360 1.164777658879757
420 1.25989159476012
480 1.110862746834755


## 精度計算

In [30]:
test_x, test_y = Variable(test_X), Variable(test_Y)
result = torch.max(model(test_x).data, 1)[1]
accuracy = sum(test_y.data.numpy() == result.numpy()) / len(test_y.data.numpy())
accuracy

0.8181818181818182

In [31]:
model(test_x).data

tensor([[-4.0034, -4.4938],
        [-4.5911, -3.7327],
        [-4.4971, -3.8576],
        [-4.1513, -4.3028],
        [-5.5246, -2.5513],
        [-3.3716, -5.3146],
        [-4.9583, -3.2646],
        [-4.7978, -3.4707],
        [-4.1268, -4.3361],
        [-5.1971, -2.9533],
        [-2.6552, -6.2517],
        [-2.1200, -6.9500],
        [-4.8956, -3.3457],
        [-5.1318, -3.0420],
        [-4.9803, -3.2372],
        [-5.1468, -3.0140],
        [-5.7749, -2.2393],
        [-5.3644, -2.7347],
        [-1.7449, -7.4463],
        [-3.0516, -5.7348],
        [-4.1974, -4.2432],
        [-5.1657, -2.9983],
        [-1.6530, -7.5661],
        [-4.1594, -4.2933],
        [-4.7556, -3.5145],
        [-4.1829, -4.2601],
        [-5.5970, -2.4602],
        [-4.8166, -3.4378],
        [-4.6001, -3.7309],
        [-5.0600, -3.1345],
        [-2.6813, -6.2116],
        [-5.1970, -2.9568],
        [-2.6673, -6.2325]])